# Project NUANS (EAI) bold text

This is the code to run the project.
In the following there are some options, like choose the best song among the available ones (link: https://drive.google.com/drive/folders/1LfL88UZII-JQziAx7Y5EO5rDXQOsA-dh?usp=sharing).
Then is possible to choose also the gpt2 model between the finetuned ones (link: https://drive.google.com/drive/folders/1umjDaVmtqEQ9LsxcdbvdYHwn1ZtLQFdt?usp=sharing)

## Import and installation

In [1]:
!pip install keybert spacy transformers textstat

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.3 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23776 sha256=c0128fccdb32447fcc0101e0d0bfdce6b4957fa08df711ecad0a1e968661e067
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for se

In [25]:
#download gpt2 finetuned
#!gdown 100qIeatL_KVBbJJjaUgCpBig63-3bPQP #gpt2 2 epochs QAstory dataset
#!gdown 1916Fcsq7JRVE7oaM0VWxwgo7nGYLfztU #gpt2 1 epoch TinyStories dataset 40000
!gdown 1nwtLMmtX0wAcRW9AUjIW1g1E9FRaUmJa #gpt2 1 epoch TinyStories dataset 80000

Downloading...
From: https://drive.google.com/uc?id=100qIeatL_KVBbJJjaUgCpBig63-3bPQP
To: /content/story_gpt2_2epochs.pt
100% 510M/510M [00:11<00:00, 42.5MB/s]


In [3]:
#download t5 finetuned
!gdown 1h4GaCcLFa7W3OWHHg6URCY3T3FZtClwh

Downloading...
From: https://drive.google.com/uc?id=1h4GaCcLFa7W3OWHHg6URCY3T3FZtClwh
To: /content/t5_gc.pt
100% 308M/308M [00:05<00:00, 51.4MB/s]


In [24]:
#download lyrics file
#!gdown 1kjwrOpypgKNh4CsO_5XSKP2iYHhpp95R #without me Eminem
#!gdown 1bCwVBKxnAc_rjTSb_GVzozxtYhDiUbTt #moves like jagger Maroon 5
#!gdown 1GKtJYQbM5GzOWB97T0gfkb7_rof9Dd8U #levitating dualipa
#!gdown 1y8jTTkoQH0CXLFK_Mwt3MNQHmht3__I0 #party in the usa
#!gdown 18NfqljJRCBg34c1rTPKBBZBfofiKeG9n #flowers
!gdown 1J6iV2RtAMhAcnIoKlbw-QJw0mzV_1erW #perfect
#!gdown 1JFrGKSFVD9Y0P5T8IH9QPqTfy1Rbg_Ro #empire state of mind

Downloading...
From: https://drive.google.com/uc?id=1y8jTTkoQH0CXLFK_Mwt3MNQHmht3__I0
To: /content/party_in_the_usa.txt
100% 2.32k/2.32k [00:00<00:00, 8.71MB/s]


In [5]:
#imports
import torch
from keybert import KeyBERT
import spacy
from transformers import pipeline, set_seed
import re
from transformers import T5Tokenizer
from transformers import GPT2Tokenizer#, GPT2LMHeadModel, GPTNeoForCausalLM
import textstat
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import Laplace

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Utility functions

In [7]:
#elaborate ner results separating date from the others and ordering them (dates are ordered by length and the other by recurrency)
def elaborate_ner_results(ner_result):
  entities = {}
  time = []
  time_word = ""
  max = -1
  people = []

  #count entities and separate "date" entities
  for ent in ner_result.ents:
    if ent.label_ == "DATE":
      time.append(ent.text)
    else:
      if ent.text not in entities:
        entities[ent.text] = 1
      else:
        entities[ent.text] += 1

  #select most long time
  for i in time:
    if len(i) > max:
      max = len(i)
      time_word = i

  #sort dictionary by value (first the more reccurrent ones)
  for i in dict(sorted(entities.items(), key=lambda item: item[1], reverse = True)):
    if "\n" in i: #if there are '\n' delete them
      words = i.split('\n')
      for k in words: #check the substring obtained
        if len(k.split(' ')) > 1:
          people.append(k)
    else: #if there isn't any '\n' check, the entity
      if len(i.split(' ')) > 1:
        people.append(i)

  return time_word, people



In [8]:
#merge keybert and ner results (removing pronouns and adding entities, and adding time in front)
def merge_data(phrases, time, entities):
  to_be = ["i'm", "i am", "you're", "you are", "he's", "he is", "she's", "she is"]
  to_have = ["i've", "i have", "you've", "you have", "he has", "she has"]
  possession_act = ["my ", "your ", "his ", "her "]
  third_person = ["she ", "he "]
  replaced = []
  results = []

  #case in which no entities are detected
  if len(time) <= 0 and len(entities) <= 0:
    for i in phrases:
      results.append(i.capitalize())
    return results

  #case in which entities are detected
  if len(entities) > 0:
    for i in phrases:
      i = i.lower()
      c = 0

      #check to_be
      for j in to_be:
        if j in i:
          if len(entities) > c:
            i = i.replace(j, entities[c] + " is")
            c += 1
          else: break

      #check to_have
      for j in to_have:
        if j in i:
          if len(entities) > c:
            i = i.replace(j, entities[c] + " has")
            c += 1
          else: break

      #check possession_act
      for j in possession_act:
        if j in i:
          if len(entities) > c:
            i = i.replace(j, entities[c] + " " )
            c += 1
          else: break

      #check possession_pass
      for j in third_person:
        if j in i:
          if len(entities) > c:
            i = i.replace(j, entities[c] + " " )
            c += 1
          else: break

      #if something change
      if c != 0:
        replaced.append(i)

  #case in which time is detected
  if len(time) > 0:
    #case in which also entities are detected
    if len(replaced) > 0:
      #merge all
      time = time.capitalize()
      for i in replaced:
        results.append(time + " " + i)
    #case in which no entities are detected
    else:
      time = time.capitalize()
      for i in phrases:
        results.append(time + " " + i)
  else:
     for i in phrases:
      results.append(i.capitalize())


  return results

In [9]:
#function to generate text (using gpt2 finetuned)
def generate_story(gpt2, tokenizer_gpt2, start, k=0, p=0.9, output_length=300, temperature=1, num_return_sequences=2, repetition_penalty=1.1):
  texts = []
  encoded_start = tokenizer_gpt2.encode(start, padding=True,truncation=True, max_length=512, return_tensors = "pt")
  output_sequences = gpt2.generate(input_ids = encoded_start, max_new_tokens = output_length, temperature = temperature, top_k = k, top_p = p, repetition_penalty = repetition_penalty, do_sample = True, num_return_sequences = num_return_sequences)
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()
  for _ , generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    # Decode text
    text = tokenizer_gpt2.decode(generated_sequence, clean_up_tokenization_spaces = True)
    # Remove all text after eos token
    text = text[: text.find(tokenizer_gpt2.eos_token)]
    #print(text, "\n")
    texts.append(text)
  return texts

In [10]:
#function to correct the grammar (using T5 finetuned)
def grammar_correction(t5, tokenizer_t5, input_text):
  # Tokenize input text
  input_text = "correcting: " + input_text
  input_ids = tokenizer_t5.encode(input_text, return_tensors='pt')
  output = t5.generate(input_ids=input_ids)
  output_text = tokenizer_t5.decode(output[0], skip_special_tokens=True)
  return output_text

In [11]:
#calculate perplexity with nltk
def calculate_perplexity(generated_text, train_frac=0.8, n=2):
    tokens = nltk.word_tokenize(generated_text.lower())
    split_idx = int(train_frac * len(tokens))
    train_tokens = tokens[:split_idx]
    test_tokens = tokens[split_idx:]

    ngrams, vocab = padded_everygram_pipeline(n, train_tokens)
    lm = Laplace(n)
    lm.fit(ngrams, vocab)
    test_ngrams = list(nltk.everygrams(test_tokens, max_len=n))
    perplexity = lm.perplexity(test_ngrams)
    return perplexity

In [12]:
#evaluation indices for the texts geenrated
def evaluation_text(texts):
  #first
  first = []
  text_generated = texts[0]
  first.append(textstat.flesch_kincaid_grade(text_generated)) #Flesch-Kincaid Grade Level: This index estimates the grade level required to understand the text.
  first.append(textstat.automated_readability_index(text_generated))  #Automated Readability Index (ARI): The ARI measures the readability of a text based on characters per word and words per sentence.
  first.append(textstat.coleman_liau_index(text_generated))  #Coleman-Liau Index: This index uses characters per word and words per sentence to estimate the grade level required to understand the text
  first.append(textstat.gunning_fog(text_generated))  #Gunning Fog Index: The Gunning Fog Index estimates the years of formal education needed to understand the text.
  first.append(textstat.smog_index(text_generated))  #SMOG Index: The SMOG index estimates the grade level required to understand the text based on the number of polysyllabic words.
  first.append(textstat.flesch_reading_ease(text_generated))
  first.append(calculate_perplexity(text_generated, train_frac=0.8, n=2))

  #second
  second = []
  text_generated = texts[1]
  second.append(textstat.flesch_kincaid_grade(text_generated)) #Flesch-Kincaid Grade Level: This index estimates the grade level required to understand the text.
  second.append(textstat.automated_readability_index(text_generated))  #Automated Readability Index (ARI): The ARI measures the readability of a text based on characters per word and words per sentence.
  second.append(textstat.coleman_liau_index(text_generated))  #Coleman-Liau Index: This index uses characters per word and words per sentence to estimate the grade level required to understand the text
  second.append(textstat.gunning_fog(text_generated))  #Gunning Fog Index: The Gunning Fog Index estimates the years of formal education needed to understand the text.
  second.append(textstat.smog_index(text_generated))  #SMOG Index: The SMOG index estimates the grade level required to understand the text based on the number of polysyllabic words.
  second.append(textstat.flesch_reading_ease(text_generated))
  second.append(calculate_perplexity(text_generated, train_frac=0.8, n=2))

  #choose
  score1 = 0
  score2 = 0
  res = []
  for i in range(0, len(first)):
    #because the 5th indice is better is is higher
    if i in [5]:
      if first[i] > second[i]:
        score1 += 1
      else:
        score2 += 1
    else:
      if first[i] > second[i]:
        score2 += 1
      else:
        score2 += 1

  #end
  if score2 > score1:
    return 1, second
  else:
    return 0, first


In [17]:
#choose best between the two candidate (already calcolated the scores)
def choose_best(first, second, txt1, txt2):
  #choose
  score1 = 0
  score2 = 0
  res = []
  for i in range(0, len(first)):
    #because the 5th indice is better is is higher
    if i in [5]:
      if first[i] > second[i]:
        score1 += 1
      else:
        score2 += 1
    else:
      if first[i] > second[i]:
        score2 += 1
      else:
        score2 += 1

  if score2 > score1:
    return second, txt2
  else:
    return first, txt1

In [18]:
#print information andscores about the text selected as the best one generated
def final_print(scores, txt):
  #unique words
  unique_words = set(txt.split())
  num_unique_words = len(unique_words)

  #avg sentence length (in words)
  sents = txt.split('.')
  avg_sentence_len_words = sum(len(x.split()) for x in sents) / len(sents)

  #avg words and sentences length (in characters)
  words = txt.split()
  sentences = txt.split(".")
  total_words = 0
  total_sentences = 0

  for word in words:
      total_words += len(word)

  for sentence in sentences:
      total_sentences += len(sentence)
  average_word_length = total_words / len(words)
  average_sentence_length = total_sentences / len(sentences)

  print("__________________________________________________________________________________")
  print("METRICS")
  print("grade_level: ", scores[0])
  print("ari: ", scores[1])
  print("coleman_liau: ", scores[2])
  print("gunning_fog: ", scores[3])
  print("smog: ", scores[4])
  print("reading_ease_score: ", scores[5])
  print("perplexity_score: ", scores[6])
  print("DATA")
  print("number unique words used: ", num_unique_words)
  print("average words length in char: ", average_word_length)
  print("avg sentences length in words: ", avg_sentence_len_words)
  print("average sentences length in char: ", average_sentence_length)
  print("__________________________________________________________________________________")
  print("TEXT OUTPUT")
  print(txt)
  print("__________________________________________________________________________________")

In [19]:
#try to delete incomplete phrases at the end
def redefine(txt):
  res = ""
  for c, i in enumerate(txt):
    if i == "\n" and len(txt[c:]) < 30: #delte the last characters only if they are less than 30 (otherwise I delete too much text)
      res = txt[:c]
  if res == "":
    return txt
  else:
    return res

## Main section

In [20]:
#pipeline function (generate stroies from lyrics)
def Stories_from_lyrics(lyrics_path, keybert, ner_spacy, t5, gpt2, tokenizer_t5, tokenizer_gpt2):
  #variables
  n_span = []
  entities = {}
  time = []
  time_word = ""
  max = -1
  people = []

  #open lyrics
  f = open(lyrics_path, "r")
  lyrics = f.read()

  #KEYBERT
  keywords = keybert.extract_keywords(lyrics, keyphrase_ngram_range=(1, 5), stop_words=None, use_maxsum=True, nr_candidates=20, top_n=2)
  for i in range(0, len(keywords)): #convert keywords extracted into string array
    n_span.append(keywords[i][0])

  #SPACY NER
  ner_result = ner(lyrics) #do ner
  time_word, people = elaborate_ner_results(ner_result) #elaborate results


  print("ner: ", people, " and time: ", time_word)

  #MERGE NER AND SPAN
  res = merge_data(n_span, time_word, people)

  print("res merg: ", res)

  #GRAMMAR CORRECTION
  corrected = []
  for i in res:
    corrected.append(grammar_correction(t5, tokenizer_t5, i))

  #GPT2
  output1 = generate_story(gpt2, tokenizer_gpt2, corrected[0])
  output2 = generate_story(gpt2, tokenizer_gpt2, corrected[1])


  #BEST TEXT CHOOSE
  index_best1, scores_best1 = evaluation_text(output1)
  index_best2, scores_best2 = evaluation_text(output2)
  #find best option between the two selected
  final_result_scores, final_result_text = choose_best(scores_best1, scores_best2, output1[index_best1], output2[index_best2])
  #cut final part not finished
  final_result_text = redefine(final_result_text)
  #print information
  final_print(final_result_scores, final_result_text)


  return final_result_text



In [54]:
#define paths

#path_gpt2 = "story_gpt2_2epochs.pt"
#path_gpt2 = "tinystory_gpt2_1epoch.pt"
path_gpt2 = "tinystory_gpt2_1epoch_v80.pt"

path_t5 = "t5_gc.pt"

#lyrics_path = "withoutme.txt"
#lyrics_path = "moves_like_jagger.txt"
#lyrics_path = "levitating.txt"
#lyrics_path = "party_in_the_usa.txt"
#lyrics_path = "flowers.txt"
lyrics_path = "perfect.txt"
#lyrics_path = "Empire_state_of_mind.txt"

In [ ]:
#LOAD MODELS

#define device
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

tokenizer_gpt2 = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token

#define tokenizer
tokenizer_t5 = T5Tokenizer.from_pretrained("t5-base")
tokenizer_t5.pad_token = tokenizer_t5.eos_token


keybert = KeyBERT()#.to(device)
ner = spacy.load('en_core_web_sm')#.to(device)
#gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')
gpt2 = torch.load(path_gpt2, map_location=torch.device(device))
t5 = torch.load(path_t5, map_location=torch.device(device))
gpt2.eval()
t5.eval()
#generator = pipeline('text-generation', model="facebook/opt-350m", do_sample=True, num_return_sequences=5).to(device)

In [55]:
res = Stories_from_lyrics(lyrics_path, keybert, ner, t5, gpt2, tokenizer_t5, tokenizer_gpt2)

ner:  []  and time:  
res merg:  ['You re holding mine baby', 'Found girl beautiful and sweet']


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


__________________________________________________________________________________
METRICS
grade_level:  5.0
ari:  6.0
coleman_liau:  6.89
gunning_fog:  6.33
smog:  8.2
reading_ease_score:  83.15
perplexity_score:  190.52279850834444
DATA
number unique words used:  168
average words length in char:  4.389105058365759
avg sentences length in words:  15.117647058823529
average sentences length in char:  81.0
__________________________________________________________________________________
TEXT OUTPUT
Found girl beautiful and sweet, she had a secret. She always wanted to have fun with her friends. One day she saw a room filled with pictures and games. She felt sad for not having enough space to explore them all.

Her friend, a boy named Jack the bear then smiled and said that it was good to spend time together and be happy. He asked if they could both play some games like this one. He pointed to a movie about dinosaurs and winked. The kids listened eagerly, and agreed to join in.
 
After